In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import backtrader as bt
import tushare as ts
from datetime import datetime
import tushare as ts

plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['figure.dpi'] = 300

In [2]:
# Create a subclass of Strategy to define the indicators and logic

class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position

In [14]:
# SMA策略

class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        self.dataclose = self.data0.close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        self.sma = bt.indicators.SimpleMovingAverage(self.data0, period = self.params.pfast)

    def next(self):
        if not self.position:  # not in the market
            if self.dataclose[0] > self.sma[0]:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.dataclose[0] < self.sma[0]:  # in the market & cross to the downside
            self.close()  # close long position

    def notify_order(self,order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm
                    )
                )
            else:
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm
                    )
                )
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('Operation Profit, Gross %.2f, Net %.2f' %
                (trade.pnl, trade.pnlcomm))

    def log(self, txt, dt=None, doprint=True):
        if doprint:
            dt = dt or self.datas[0].datatime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))


In [ ]:
# Create a subclass of Strategy to define the indicators and logic

class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.order_target_size(target=1)  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.order_target_size(target=0)  # close long position

In [ ]:
# Create a subclass of SignaStrategy to define the indicators and signals

class SmaCross(bt.SignalStrategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
        self.signal_add(bt.SIGNAL_LONG, crossover)  # use it as LONG signal

In [ ]:
pro = ts.pro_api('5d3c09f16bc3ad409898ff6cfc10aadeeae28ba23aa240c047dd125c')

def get_data(code='600519.SH',starttime='20170101',endtime='20200101'):
    df = pro.daily(ts_code = code,start_date = starttime,end_date = endtime)
    df.index = pd.to_datetime(df.trade_date)
    df['openinterest'] = 0
    df = df[['open','high','low','close','vol','openinterest']]
    df = df.iloc[::-1]  # 反转数据
    df= df.rename(columns={'vol':'volume'})
    return df

stock_df = get_data()
fromdate = datetime(2017,1,1)
todate = datetime(2020,1,1)
# 加载并读取数据源 dataname(数据来源) fromdate(date格式) todate
data = bt.feeds.PandasData(dataname=stock_df,fromdate = fromdate,todate = todate)

              open    high     low   close    volume  openinterest
trade_date                                                        
2017-01-03  334.28  337.00  332.81  334.56  20763.89             0
2017-01-04  334.62  352.17  334.60  351.91  65257.38             0
2017-01-05  350.00  351.45  345.44  346.74  41704.48             0
2017-01-06  346.64  359.78  346.10  350.76  68095.62             0
2017-01-09  347.80  352.88  346.54  348.51  35405.00             0
              open    high     low   close    volume  openinterest
trade_date                                                        
2017-01-03  334.28  337.00  332.81  334.56  20763.89             0
2017-01-04  334.62  352.17  334.60  351.91  65257.38             0
2017-01-05  350.00  351.45  345.44  346.74  41704.48             0
2017-01-06  346.64  359.78  346.10  350.76  68095.62             0
2017-01-09  347.80  352.88  346.54  348.51  35405.00             0


In [ ]:
cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

# Create a data feed
cerebro.adddata(data)  # Add the data feed

cerebro.addstrategy(SmaCross)  # Add the trading strategy

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DrawDown')

cerebro.broker.setcash(1000000)
cerebro.broker.setcommission(commission=0.0005)

cerebro.addsizer(bt.sizers.PercentSizer, percents=90)
cerebro.run()  # run it all

In [17]:
cerebro.plot(mpl={'style': 'mpl'})[0][0]

ValueError: Axis limits cannot be NaN or Inf

In [18]:
cerebro.plot()

ValueError: Axis limits cannot be NaN or Inf